<a href="https://colab.research.google.com/github/zabi076/Amazon-Clone/blob/main/paperwala.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install langchain langgraph sentence-transformers chromadb fpdf
!pip install --upgrade langgraph

In [18]:
from langchain.prompts import PromptTemplate
import Langgraph, State, Node,Edge from langgraph.graph
from sentence_transformers import SentenceTransformer
from chromadb import Client
from fpdf import FPDF

# Initialize the vector database and model
client = Client()  # Assumes the vector DB is set up (e.g., ChromaDB)
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Question Paper Generator Workflow
class QuestionPaperGenerator(LangGraph):
    def __init__(self):
        super().__init__()

        # Add state for storing inputs and intermediate results
        self.state = State()

        # Define nodes
        self.add_node("InputNode", self.input_node)
        self.add_node("FetchQuestionsNode", self.fetch_questions_node)
        self.add_node("FilterQuestionsNode", self.filter_questions_node)
        self.add_node("GeneratePaperNode", self.generate_paper_node)
        self.add_node("OutputNode", self.output_node)

        # Define edges
        self.add_edge(Edge("InputNode", "FetchQuestionsNode"))
        self.add_edge(Edge("FetchQuestionsNode", "FilterQuestionsNode"))
        self.add_edge(Edge("FilterQuestionsNode", "GeneratePaperNode"))
        self.add_edge(Edge("GeneratePaperNode", "OutputNode"))

    def input_node(self, state: State):
        """
        Takes user inputs: subject, level, total marks, and structure.
        """
        # Example: Inputs received from a UI
        user_inputs = {
            "subject": "Mathematics",
            "level": "O Level",
            "total_marks": 100,
            "structure": {"MCQs": 30, "Short": 40, "Long": 30},
        }
        state.update(user_inputs)

    def fetch_questions_node(self, state: State):
        """
        Retrieves questions from the vector database using the subject and level.
        """
        subject = state["subject"]
        level = state["level"]

        # Query the vector database for relevant questions
        query = f"Retrieve {level} {subject} questions"
        embeddings = embedding_model.encode(query)
        results = client.query(embedding=embeddings, top_k=100)  # Fetch top 100 questions
        state["questions"] = results

    def filter_questions_node(self, state: State):
        """
        Filters and organizes questions based on the paper structure.
        """
        structure = state["structure"]
        questions = state["questions"]

        filtered_questions = {
            "MCQs": [q for q in questions if q["type"] == "MCQ"][: structure["MCQs"] // 2],
            "Short": [q for q in questions if q["type"] == "Short"][: structure["Short"] // 5],
            "Long": [q for q in questions if q["type"] == "Long"][: structure["Long"] // 10],
        }
        state["filtered_questions"] = filtered_questions

    def generate_paper_node(self, state: State):
        """
        Formats the selected questions into a structured question paper.
        """
        filtered_questions = state["filtered_questions"]
        total_marks = state["total_marks"]

        # Generate paper text
        paper = f"Question Paper\nTotal Marks: {total_marks}\n\n"
        for section, questions in filtered_questions.items():
            paper += f"Section: {section}\n"
            for i, question in enumerate(questions, 1):
                paper += f"Q{i}. {question['text']} [{question['marks']} Marks]\n"
            paper += "\n"
        state["paper_text"] = paper

    def output_node(self, state: State):
        """
        Outputs the question paper as a PDF or text file.
        """
        paper_text = state["paper_text"]

        # Export as PDF
        pdf = FPDF()
        pdf.add_page()
        pdf.set_font("Arial", size=12)
        pdf.multi_cell(0, 10, paper_text)
        pdf.output("question_paper.pdf")
        print("Question paper saved as 'question_paper.pdf'.")



SyntaxError: invalid syntax (<ipython-input-18-c91e9f1dafde>, line 2)

In [19]:
if __name__ == "__main__":
    qp_generator = QuestionPaperGenerator()
    qp_generator.run()  # This runs the entire workflow from InputNode to OutputNode


NameError: name 'QuestionPaperGenerator' is not defined

In [ ]:
[{"text": "Solve for x: 2x + 3 = 7", "type": "MCQ", "marks": 2}, ...]
